In [1286]:
import math
from numba import njit, jit, prange
import numpy as np
import pandas as pd
import requests
from scipy.optimize import minimize
import numpy as np
import pandas as pd
from typing import Tuple, Optional, Dict, List
import warnings
warnings.filterwarnings('ignore')

### Preparation orderbook data

In [567]:
@jit(nopython=True)
def calculate_log_returns(prices):
    n = len(prices)
    log_ret = np.zeros(n)
    for i in range(1, n):
        log_ret[i] = np.log(prices[i] / prices[i-1])
    return log_ret

def lob_resampling(data, sampling_freq='100ms'):
    data['local_timestamp'] = pd.to_datetime(data['local_timestamp'], unit='us')
    data = data.set_index('local_timestamp').sort_index()
    data['mid_price'] = (data['bids[0].price'] + data['asks[0].price']) / 2
    data['spread'] = data['asks[0].price'] - data['bids[0].price']
    data_resampled = data.asfreq(sampling_freq, method='ffill').dropna()
    prices = data_resampled['mid_price'].values
    data_resampled['log_ret'] = calculate_log_returns(prices)
    
    return data_resampled.iloc[1:] 

### Parsing trades

In [619]:
def get_historical_trades(symbol, start_time=None, end_time=None, limit=1000):
    url = "https://api.binance.com/api/v3/aggTrades"
    params = {
        'symbol': symbol,
        'limit': limit,
    }
    if start_time:
        params['startTime'] = int(pd.Timestamp(start_time).timestamp() * 1000)
    if end_time:
        params['endTime'] = int(pd.Timestamp(end_time).timestamp() * 1000)
    
    response = requests.get(url, params=params)
    trades = response.json()
    df = pd.DataFrame(trades)
    df['time'] = pd.to_datetime(df['T'], unit='ms')
    df['price'] = df['p'].astype(float)
    df['volume'] = df['q'].astype(float)
    return df[['time', 'price', 'volume', 'f', 'l', 'm']]


def fetch_all_trades(symbol, start_time, end_time):
    all_trades = []
    current_time = pd.Timestamp(start_time)
    end_time = pd.Timestamp(end_time)
    
    while current_time < end_time:
        chunk = get_historical_trades(
            symbol=symbol,
            start_time=current_time,
            end_time=current_time + pd.Timedelta(hours=1),  
            limit=1000
        )
        if not chunk.empty:
            all_trades.append(chunk)
            current_time = chunk['time'].iloc[-1] + pd.Timedelta(milliseconds=1)
        else:
            current_time += pd.Timedelta(hours=1)
    
    return pd.concat(all_trades)


### Test data - SOL orderbook & trades

In [1178]:
sol_lob = pd.read_parquet(r'C:\Users\Эвелина Новикова\Documents\Downloads\binance-futures_book_snapshot_25_2024-11-01_SOLUSDT.parquet')

In [543]:
sol_trades = fetch_all_trades('SOLUSDT', '2024-11-01 00:00:00', '2024-11-01 23:59:59')

### Preparation trades 

In [813]:
def trades_resampling(resampled_lob, trades):
    trades = trades.rename(columns={'time': 'local_timestamp'})
    
    buy = trades[trades['m'] == False]
    sell = trades[trades['m'] == True]
    
    buy = buy.set_index('local_timestamp')
    sell = sell.set_index('local_timestamp')

    resampled_lob_copy = resampled_lob.copy()
    resampled_lob_copy = resampled_lob_copy.reset_index()
    start_agg = resampled_lob_copy.iloc[0, 0]
    buy_res = buy.resample("100ms", origin=start_agg).agg({
    "price": "max"}).fillna(-np.inf)
    sell_res = sell.resample("100ms", origin=start_agg).agg({
    "price": "max"}).fillna(-np.inf)

    return buy_res, sell_res

### Dynamic calibrating A & k

In [903]:
def delta_grid(resampled_lob, resampled_side, N1=7, N2=4):
    joined = resampled_side.join(resampled_lob, how='inner')
    not_null = joined[joined['price'] > 0]
    not_null['distance'] = np.abs(not_null['price'] - not_null['mid_price'])
    not_null['delta'] = not_null['distance']/not_null['mid_price'] * 100

    median = np.median(not_null['delta'])
    q90 = np.quantile(not_null['delta'], 0.9)
    max = np.max(not_null['delta'])

    # N_1 точек в [min_delta, q90] (линейно)
    # N_2 точек в (q90, max_d] (лог)
    
    close = np.linspace(median * 0.25, q90, N_1, endpoint=True)
    far = np.exp(np.linspace(np.log(q90 * 1.0001), np.log(max), N_2, endpoint=True))
    grid = np.unique(np.concatenate([close, far]))

    return grid

In [885]:
def compute_time_to_fill(arr_mid, trades_prices, delta, tau_steps, chosen_side):
    N = arr_mid.shape[0]
    orders = np.empty(N, dtype=np.float64)
    for i in range(N):
        if chosen_side == 'ask':
            orders[i] = arr_mid[i] * (1.0 + delta*0.01)
        elif chosen_side == 'bid':
            orders[i] = arr_mid[i] * (1.0 - delta*0.01)
        else:
            break
            print('Unknown side')
    
    ttf = np.full(N, tau_steps, dtype=np.int32)
    for j in range(N):
        p = trades_prices[j]
        if p <= 0.0:
            continue
            
        start = j - tau_steps
        if start < 0:
            start = 0

        for t in range(start, j+1):
            if chosen_side == 'ask' and orders[t] <= p or chosen_side == 'bid' and orders[t] >= p:
                cand = j - t
                if cand < ttf[t]:
                    ttf[t] = cand
    fills = 0
    for i in range(N):
        if ttf[i] < tau_steps:
            fills += 1

    return ttf, fills

In [947]:
def calc_intensity(lob, trades, tau_seconds=5, step_seconds=0.1):
    resampled_lob = lob_resampling(lob)
    mid = resampled_lob['mid_price'].values.astype(np.float64)
    buy_res, sell_res = trades_resampling(resampled_lob, trades)
    buy = buy_res['price'].values.astype(np.float64)
    sell = sell_res['price'].values.astype(np.float64)
    deltas_ask = delta_grid(resampled_lob, buy_res)
    deltas_bid = delta_grid(resampled_lob, sell_res)
    tau_steps = int(round(tau_seconds / step_seconds))
    
    intensity_ask = {}
    intensity_bid = {}
    
    for delta in deltas_ask:
        ttf_steps, fills = compute_time_to_fill(mid, buy, float(delta), tau_steps, 'ask')
        intensity_ask[f'{delta}'] = [ttf_steps, fills]
    for delta in deltas_bid:
        ttf_steps, fills = compute_time_to_fill(mid, sell, float(delta), tau_steps, 'bid')
        intensity_bid[f'{delta}'] = [ttf_steps, fills]
        
    return intensity_ask, intensity_bid

In [1238]:
def calibrate_A_k(side_data, dt=0.1, tau=5.0, min_points=3):
    
    tau_steps = int(tau / dt)

    deltas = []
    lambdas = []

    for delta_str, (ttf_arr, fills) in side_data.items():
        delta = float(delta_str)
        ttf = np.array(ttf_arr, dtype=np.int32)
        fills = int(fills)

        if ttf.size == 0:
            continue
            
        exp_time = np.minimum(ttf, tau_steps).sum() * dt

        if exp_time <= 0:
            continue

        lam = fills / exp_time

        if lam <= 0:
            continue

        deltas.append(delta)
        lambdas.append(lam)

    deltas = np.array(deltas)
    lambdas = np.array(lambdas)

    if len(deltas) < min_points:
        raise ValueError("Not enough points for regression")

    log_lambdas = np.log(lambdas)

    slope, intercept = np.polyfit(deltas, log_lambdas, 1)

    k = -slope
    A = np.exp(intercept)

    return A, k

### Dynamic calibrating sigma

In [1206]:
class HARVolatilityPredictor:
    """
    HAR-like model for predicting volatility on high-frequency data
    """
    
    def __init__(self, lob: pd.DataFrame):
        resampled_lob = lob_resampling(lob)
        self.midprice = resampled_lob['mid_price'].values
        self.returns = np.diff(np.log(self.midprice))  # лог-ретурны
    
    def calculate_har_volatility(self):
        """
        Three types of volatility:
        - short-term: 10 minutes
        - medium-term: 30 minutes
        - long-term: 1 hour
        """
        rv_array = self.returns ** 2
        
        rv_short = np.mean(rv_array[-6000:]) if len(rv_array) >= 6000 else np.mean(rv_array)
        rv_medium = np.mean(rv_array[-18000:]) if len(rv_array) >= 18000 else rv_short
        rv_long = np.mean(rv_array[-36000:]) if len(rv_array) >= 36000 else rv_medium
        
        return rv_short, rv_medium, rv_long
    
    def predict_volatility(self):
        """
        Volatility Predictions
        """
        rv_short, rv_medium, rv_long = self.calculate_har_volatility()
        sigma_har = 0.5 * rv_short + 0.3 * rv_medium + 0.2 * rv_long

        return sigma_har


### Collecting config

In [1256]:
def make_config(lob, trades, gamma):
    
    intensity_ask, intensity_bid = calc_intensity(lob, trades)
    k_bid, A_bid = calibrate_A_k(intensity_bid)
    k_ask, A_ask = calibrate_A_k(intensity_ask)
    A = (A_ask * A_bid)**0.5
    k = (k_ask + k_bid)/2

    predictor = HARVolatilityPredictor(lob)
    sigma = predictor.predict_volatility()

    mid = (lob['bids[0].price'] + lob['asks[0].price']) / 2
    initial_price = mid[0]

    T = 1.0
    dt = 0.1

    return {'T': T,
    'dt': dt,
    'sigma': sigma,
    'initial_price': initial_price,
    'A': A,
    'k': k,
    'gamma': gamma}

### Strategy simulation

In [1262]:
class AS_Strategy:
    """
    Simulate AS strategy
    """

    def __init__(self, config: Dict):
        self.config = config
        self.T = config['T']
        self.dt = config['dt']
        self.sigma = config['sigma']
        self.s0 = config['initial_price']
        self.A = config['A']
        self.k = config['k']
        self.n_steps = int(self.T / self.dt)

    def calculate_reservation_prices(self, s: float, q: int, t: float, gamma: float) -> Tuple[float, float, float]:
        """
        Calculate reservation prices for bid and ask
        """
        time_to_expiry = self.T - t

        r = s - q * gamma * self.sigma**2 * time_to_expiry
        r_bid = s + (-1 - 2*q) * gamma * self.sigma**2 * time_to_expiry / 2
        r_ask = s + (1 - 2*q) * gamma * self.sigma**2 * time_to_expiry / 2

        return r, r_bid, r_ask

    def calculate_optimal_spread(self, gamma: float, t: float) -> float:
        """
        Calculate optimal total spread
        """
        time_to_expiry = self.T - t

        if time_to_expiry <= 0:
            return 0.0

        if gamma <= 0:
            return 0.0

        try:
            spread = gamma * self.sigma**2 * time_to_expiry + (2/gamma) * np.log(1 + gamma/self.k)
        except (OverflowError, ZeroDivisionError):
            spread = gamma * self.sigma**2 * time_to_expiry

        return spread

    def calculate_order_intensity(self, delta: float) -> float:
        """
        Calculate order arrival intensity
        """
        if delta <= 0:
            return self.A

        try:
            intensity = self.A * np.exp(-self.k * delta)
        except OverflowError:
            intensity = 0.0

        return intensity
        
    def price_path(self, lob: pd.DataFrame) -> np.ndarray:
        """
        Mid-price preprocessing
        """
        resampled_lob = lob_resampling(lob)
        price_path = resampled_lob['mid_price'].values

        return price_path

    def simulate_strategy(self, strategy_type: str, gamma: float, lob: pd.DataFrame) -> Dict:
        """
        Simulate a single path for given strategy
        """
        price_path = self.price_path(lob)
        inventory = 0
        cash = 0.0

        inventory_history = []
        cash_history = []
        spread_history = []

        for i in range(self.n_steps):
            t = i * self.dt
            s = price_path[i]
            r, r_bid, r_ask = self.calculate_reservation_prices(s, inventory, t, gamma)
            total_spread = self.calculate_optimal_spread(gamma, t)

            # Set quotes based on strategy
            if strategy_type == 'inventory':
                bid_price = r - total_spread / 2
                ask_price = r + total_spread / 2
            elif strategy_type == 'symmetric':
                bid_price = s - total_spread / 2
                ask_price = s + total_spread / 2
            else:
                raise ValueError(f"Unknown strategy type: {strategy_type}")

            delta_bid = s - bid_price
            delta_ask = ask_price - s

            delta_bid = delta_bid
            delta_ask = delta_ask

            lambda_bid = self.calculate_order_intensity(delta_bid)
            lambda_ask = self.calculate_order_intensity(delta_ask)

            # Simulate order arrivals (Poisson process)
            prob_bid = min(lambda_bid * self.dt, 1.0)
            prob_ask = min(lambda_ask * self.dt, 1.0)

            bid_order = np.random.random() < prob_bid
            ask_order = np.random.random() < prob_ask

            if bid_order:
                inventory += 1
                cash -= bid_price

            if ask_order:
                inventory -= 1
                cash += ask_price

            inventory_history.append(inventory)
            cash_history.append(cash)
            spread_history.append(delta_bid + delta_ask)

        # Final P&L calculation
        final_price = price_path[-1]
        final_pnl = cash + inventory * final_price

        return {
            'final_pnl': final_pnl,
            'final_inventory': inventory,
            'final_cash': cash,
            'final_price': final_price,
            'mean_spread': np.mean(spread_history),
            'inventory_std': np.std(inventory_history)
        }

In [1284]:
# config = make_config(sol_lob, sol_trades, 0.01)
# simulator = AS_Strategy(config)
# result = simulator.simulate_strategy(strategy_type='symmetric', gamma=0.1, lob=sol_lob)